<sup>This notebook is adapted from code from *Deep Learning with Python 2nd edition* by Francois Challot. 

<sup>See the original code in the books' companion [GitHub](https://github.com/fchollet/deep-learning-with-python-notebooks).  Find the book here: [Amazon Link](https://www.amazon.com/Learning-Python-Second-Fran-C3-A7ois-Chollet-dp-1617296864/dp/1617296864/)


### Keras CNN Using Functional API

Doing some things differently for demonstration purposes. This notebook also discusses challenges of smaller data sets, data augmentation, feature extraction with a pretrained model, and fine-tuning a pretrained model. 

#### Using the Cats and Dogs data set

Data from a Kaggle data set with 5K images of cats and dogs. To get the data:

* Sign in, or get a free Kaggle account, you can sign in with Google
* Go to: https://www.kaggle.com/c/dogs-vs-cats/overview
* Go to the data tab and download train.zip and test.zip
* You will be prompted to accept terms of competition
* Unzip in the folder of your choice
* Then upload to Colab following the steps below


In [3]:
# upload train.zip

from google.colab import files
files.upload()

Saving train.zip to train.zip


KeyboardInterrupt: ignored

### Load the data

Load only the training data for this notebook, then divide into train, validate, test. 

In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

### The network architecture

Build the model with one hidden layer of 512 nodes. The output layer has 10 nodes, one for each digit 0-9. Softmax activation spreads the probability among the 10 nodes so that the most likely digit has the highest probability.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0     

**The compilation step**

In [ ]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

### Fit the model

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 56s 58ms/step - loss: 0.1580 - accuracy: 0.9505
Epoch 2/5
938/938 [==============================] - 55s 58ms/step - loss: 0.0461 - accuracy: 0.9862
Epoch 3/5
938/938 [==============================] - 55s 59ms/step - loss: 0.0319 - accuracy: 0.9904
Epoch 4/5
938/938 [==============================] - 56s 60ms/step - loss: 0.0233 - accuracy: 0.9926
Epoch 5/5
938/938 [==============================] - 56s 60ms/step - loss: 0.0190 - accuracy: 0.9941


**Evaluating the model on new data**

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")

313/313 [==============================] - 3s 10ms/step - loss: 0.0342 - accuracy: 0.9893
test_acc: 0.989300012588501
